In [53]:
import random
random.seed(42)
from collections import deque

# Lit 🔥 Review Visualiser

# THE PAPER IS THE GRAPH

what does that mean? Welllll a Paper is an inherenetly recursive object in the real world, it has references which are papers which have references which are papers... 

So the object that we create to represent a Paper needs to represent this property, however we need to add a base case (a depth at which we no longer care about the references)

The real world has no recursion depth limits but python does

In [58]:
class Paper:

    def __init__(self, name = 0, distance = 0):
        """this constructor is recursive, eaech Paper create 
        a list of Papers that it references"""
        self.__name = name
        self.__distance = distance
        self.__references = self.create_references()

    def create_references(self):
        # return an empty list if we have exceeded relevant depth
        if self.get_distance() > 3:
            return []
        # otherwise, return a list of Paper objects, their distance set to this paper's distance + 1
        return [Paper(self.get_name() - n, self.get_distance() + 1) for n in range(1, 6)]
    
    def find_distance(self, name):
        queue = deque([(self, 0)])  # the queue will hold tuples of (paper, distance)
        while queue:
            paper, distance = queue.popleft()  # Dequeue a paper

            # if the target paper is this paper return the distance
            if name == paper.get_name():
                return distance
            
            # if the target paper is found in a paper's references, return the distance + 1
            elif name in [reference.get_name() for reference in paper.get_references()]:
                    print(f"Found {name} in {paper.get_name()}'s references")
                    return distance + 1
            else:
                # add all the paper's references to the queue with a distance of distance + 1
                for reference in paper.get_references():
                    queue.append((reference, distance + 1))

        # if the queue is empty and the paper hasn't been found, return infinity
        return float('inf')
    
    def get_edges(self, name):
        # NOTE can you think of good stop case?
        # i.e. a natural time to stop the recursion?
        if self.get_name() == name:
            return 1
        else:
            return sum([reference.get_edges(name) for reference in self.get_references()])
        
    
    def get_name(self):
        return self.__name
    
    def get_distance(self):
        return self.__distance

    def get_references(self):
        return self.__references

    def __repr__(self):
        return f"{self.get_name()}: {self.get_references()}"

In [59]:
paper = Paper(2024)

In [63]:
paper.get_edges(2010)

71

In [3]:
paper.get_references()

[2023: [2022: [2021: [2020: [], 2019: [], 2018: [], 2017: [], 2016: []], 2020: [2019: [], 2018: [], 2017: [], 2016: [], 2015: []], 2019: [2018: [], 2017: [], 2016: [], 2015: [], 2014: []], 2018: [2017: [], 2016: [], 2015: [], 2014: [], 2013: []], 2017: [2016: [], 2015: [], 2014: [], 2013: [], 2012: []]], 2021: [2020: [2019: [], 2018: [], 2017: [], 2016: [], 2015: []], 2019: [2018: [], 2017: [], 2016: [], 2015: [], 2014: []], 2018: [2017: [], 2016: [], 2015: [], 2014: [], 2013: []], 2017: [2016: [], 2015: [], 2014: [], 2013: [], 2012: []], 2016: [2015: [], 2014: [], 2013: [], 2012: [], 2011: []]], 2020: [2019: [2018: [], 2017: [], 2016: [], 2015: [], 2014: []], 2018: [2017: [], 2016: [], 2015: [], 2014: [], 2013: []], 2017: [2016: [], 2015: [], 2014: [], 2013: [], 2012: []], 2016: [2015: [], 2014: [], 2013: [], 2012: [], 2011: []], 2015: [2014: [], 2013: [], 2012: [], 2011: [], 2010: []]], 2019: [2018: [2017: [], 2016: [], 2015: [], 2014: [], 2013: []], 2017: [2016: [], 2015: [], 2014: 

`find_distance()` returns the shortest distance found from one Paper to another

In [70]:
paper.find_distance(2024)

0

Distance from the paper to itself is 0

In [71]:
paper.find_distance(2020)

Found 2020 in 2024's references


1

In [72]:
paper.find_distance(2022)

Found 2022 in 2024's references


1

The search is implemented breadth-first: you can see 2022 in 2023's references above, but we check all the references of a given paper _before_ we check _its_ references

Attempting to find the distance to a paper that isn't referenced will return `inf`, in other words, not in the references

In [75]:
paper.find_distance(2025)

inf

In [76]:
paper.find_distance(1997)

inf

## Testing